In [2]:
import pandas as pd

In [3]:
df_init = pd.read_csv("train.csv")
df = df_init

In [4]:
from ydata_profiling import ProfileReport

In [5]:
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_file("report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|███████████████████████████████████████████| 18/18 [00:00<00:00, 35.73it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
df.sample(5)

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,WeightCategory
10157,10157,Female,21.928973,1.644560,49.657270,yes,yes,1.344854,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,Sometimes,Public_Transportation,Insufficient_Weight
4009,4009,Female,20.000000,1.650000,80.000000,yes,no,2.000000,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,no,Public_Transportation,Overweight_Level_II
15146,15146,Male,29.906575,1.947406,123.721352,yes,yes,2.002784,3.0,Sometimes,yes,2.277937,no,0.756277,0.000000,Sometimes,Automobile,Obesity_Type_II
2895,2895,Female,22.977357,1.603501,82.000000,yes,yes,1.834155,1.0,Sometimes,no,2.000000,no,2.971832,0.000000,no,Public_Transportation,Obesity_Type_I
3875,3875,Male,30.958051,1.726279,125.064264,yes,yes,2.499108,3.0,Sometimes,no,1.818026,no,0.627700,0.402075,Sometimes,Public_Transportation,Obesity_Type_II


In [7]:
#BMI - weight / heights ^ 2
#df['BMI'] = df['Weight'] / (df['Height'] ** 2)

In [8]:
print(df.columns.tolist())

['id', 'Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'WeightCategory']


In [9]:
"""
FAVC - Frequent High Calory Food consumption
FCVC	Frequency of consumption of vegetables	 -- Numeric scale (1–3)	1 = rarely, 3 = often — higher = healthier diet
NCP	Number of main meals per day	Numeric (1–4)	Typically 3 = normal, fewer/more can affect metabolism
CAEC	Consumption of food between meals (snacking)	no, Sometimes, Frequently, Always	Indicates snacking habits
SMOKE	Whether the person smokes	yes / no	Smoking affects metabolism & appetite
CH2O	Daily water intake (liters)	Numeric scale (1–3)	1 = <1L, 2 = 1–2L, 3 = >2L
SCC	Calories consumption monitoring	yes / no	Whether person tracks calorie intake
FAF	Physical activity frequency	Numeric scale (0–3)	0 = none, 3 = active almost every day
TUE	Time using electronic devices per day	Numeric scale (0–2 or 1–3)	0 = <2h, 2 = >8h — sedentary indicator
CALC	Alcohol consumption frequency	no, Sometimes, Frequently, Always	Alcohol adds empty calories
MTRANS	Main mode of transportation	Walking, Bike, Public_Transportation, Automobile	Indicates daily activity level
WeightCategory	Target label — category of weight	e.g. Normal_Weight, Overweight, Obesity_Type_I, etc.	Classification target
"""

'\nFAVC - Frequent High Calory Food consumption\nFCVC\tFrequency of consumption of vegetables\t -- Numeric scale (1–3)\t1 = rarely, 3 = often — higher = healthier diet\nNCP\tNumber of main meals per day\tNumeric (1–4)\tTypically 3 = normal, fewer/more can affect metabolism\nCAEC\tConsumption of food between meals (snacking)\tno, Sometimes, Frequently, Always\tIndicates snacking habits\nSMOKE\tWhether the person smokes\tyes / no\tSmoking affects metabolism & appetite\nCH2O\tDaily water intake (liters)\tNumeric scale (1–3)\t1 = <1L, 2 = 1–2L, 3 = >2L\nSCC\tCalories consumption monitoring\tyes / no\tWhether person tracks calorie intake\nFAF\tPhysical activity frequency\tNumeric scale (0–3)\t0 = none, 3 = active almost every day\nTUE\tTime using electronic devices per day\tNumeric scale (0–2 or 1–3)\t0 = <2h, 2 = >8h — sedentary indicator\nCALC\tAlcohol consumption frequency\tno, Sometimes, Frequently, Always\tAlcohol adds empty calories\nMTRANS\tMain mode of transportation\tWalking, Bike,

In [10]:
outputs = list(df['WeightCategory'].unique())
outputs

['Overweight_Level_II',
 'Normal_Weight',
 'Insufficient_Weight',
 'Obesity_Type_III',
 'Obesity_Type_II',
 'Overweight_Level_I',
 'Obesity_Type_I']

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

class BMICalculatorAndDropper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['BMI'] = X['Weight'] / (X['Height'] ** 2)
        X = X.drop(columns=['Weight', 'Height','id'])
        return X

In [12]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
binary_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']
ordinal_cols = ['FCVC', 'CAEC', 'CH2O', 'FAF', 'TUE', 'CALC']
nominal_cols = ['MTRANS']
numeric_cols = ['Age', 'NCP', 'BMI']

preprocessor = ColumnTransformer([
    ('binary', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), binary_cols),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ordinal_cols),
    ('nominal', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), nominal_cols),
    ('numeric', StandardScaler(), numeric_cols)
])


In [13]:
data = df.drop(columns = ['WeightCategory'])
y = df['WeightCategory']

In [14]:
#data_transformed= preprocessor.fit_transform(data)

In [15]:
#Random Forest
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

model = Pipeline([
    ('bmi_dropper', BMICalculatorAndDropper()),
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=200, random_state=42))
])


In [16]:
model.fit(data, y)

,steps,"[('bmi_dropper', ...), ('preprocessor', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('binary', ...), ('ordinal', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [17]:
test_df = pd.read_csv('test.csv')
y_pred = model.predict(test_df)

In [18]:
submission = pd.DataFrame({
    'id': test_df['id'],
    'PredictedWeightCategory': y_pred
})
submission.to_csv('submission.csv', index=False)


In [19]:
#XGBOOST
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

xgb_model = XGBClassifier(
    use_label_encoder=False, 
    eval_metric='mlogloss',  
    random_state=42,
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1
)
model_pipeline = Pipeline([
    ('bmi_dropper', BMICalculatorAndDropper()),
    ('preprocessor', preprocessor),
    ('classifier', xgb_model)
])

In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_enc = le.fit_transform(y)
y_test_enc = le.transform(y)  
model_pipeline.fit(data, y_train_enc)
y_pred_enc = model_pipeline.predict(test_df)
y_pred_labels = le.inverse_transform(y_pred_enc)


/home/dharani-prasad-s/jupyter_env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:32:52] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [21]:
submission = pd.DataFrame({
    'id': test_df['id'],
    'PredictedWeightCategory': y_pred_labels
})
submission.to_csv('submission2_new.csv', index=False)

In [22]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#Grid search
le = LabelEncoder()
y_train_enc = le.fit_transform(y)
pipeline = Pipeline([
    ('bmi_and_drop', BMICalculatorAndDropper()),
    ('preprocessor', preprocessor), 
    ('classifier', XGBClassifier(eval_metric='mlogloss', random_state=42,verbosity = 3))
])
param_grid = {
    'classifier__max_depth': [4, 6, 8],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__n_estimators': [100, 200],
    'classifier__subsample': [0.7, 0.9, 1.0],
    'classifier__colsample_bytree': [0.7, 1.0]
}
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)
grid_search.fit(data, y_train_enc)

print("Best params:", grid_search.best_params_)
print("Best CV accuracy:", grid_search.best_score_)
y_pred_enc = grid_search.best_estimator_.predict(test_df)
y_pred_labels = le.inverse_transform(y_pred_enc)


Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [ ]:
bmi_transformer = BMICalculatorAndDropper()
data_transformed = bmi_transformer.fit_transform(test_df)
test_data_preprocessed = preprocessor.fit_transform(data_transformed)

In [36]:
y_test_pred_enc = best_model.predict(test_data_preprocessed)

y_test_pred_labels = le.inverse_transform(y_test_pred_enc)

In [39]:
submission = pd.DataFrame({
    'id': test_df['id'],
    'PredictedWeightCategory': y_pred_labels
})
submission.to_csv('submissiongrid.csv', index=False)